This attempt will be using only non nn classifiers

In [10]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [5]:
df = pd.read_csv('../datasets/cleaned/combined_text.csv')

In [6]:
df.head()

,combined_text,category,asin
0,I have a 9 year old Badger 1 that needs replac...,appliances,B00004U9JP
1,model number This may help InSinkErator Model ...,appliances,B00004U9JP
2,can I replace Badger 1 1/3 with a Badger 5 1/2...,appliances,B00004U9JP
3,Does this come with power cord and dishwasher ...,appliances,B00004U9JP
4,loud noise inside when turned on. sounds like ...,appliances,B00004U9JP


In [11]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])